# imports

In [17]:
import pandas as pd
import numpy as np

In [4]:
wefarm = pd.read_csv(r"https://producersdirect-backups.s3.eu-west-1.amazonaws.com/athena-results/Unsaved/2025/09/26/b0cd514b-b9cc-4972-a0c2-c91726e6d825.csv")

# peek at the data

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)

wefarm.head(5)

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,response_topic,response_sent,question_user_type,question_user_status,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,NaN,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',NaN,2019-01-24 17:54:06.216221+00,farmer,live,ug,NaN,NaN,2017-11-18 13:09:11+00,farmer,live,ug,NaN,NaN,2017-05-09 09:19:33+00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,NaN,2018-01-04 08:57:28+00,farmer,live,ug,NaN,NaN,2017-11-20 11:55:48+00,farmer,zombie,ug,NaN,NaN,2017-11-22 10:13:03+00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,cattle,2017-11-22 12:35:26+00,farmer,zombie,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00


In [6]:
wefarm.shape

(20304843, 24)

# install libraries

In [7]:
!pip install gensim pyldavis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 44.1 MB/s eta 0:00:00


In [8]:
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim import models
import gensim

from nltk.stem import WordNetLemmatizer

import pyLDAvis.gensim_models

from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

# create dataframes by language

In [11]:
wefarm_eng_df = wefarm[wefarm["question_language"] == "eng"][["question_id", "question_content"]].drop_duplicates()

# preprocess text

In [12]:
lem = WordNetLemmatizer()

def preprocess_text(text):
  tokens = simple_preprocess(text, deacc=True)
  return [lem.lemmatize(text, pos='v') for text in tokens if text not in STOPWORDS]

In [13]:
processed_questions_eng_lst = []
for question in tqdm(wefarm_eng_df["question_content"]):
  processed_questions_eng_lst.append(preprocess_text(question))

100%|██████████| 2940542/2940542 [01:12<00:00, 40520.84it/s]


# model

In [14]:
processed_questions_eng_gensim = gensim.corpora.Dictionary(processed_questions_eng_lst)
bow_corpus = []
for doc in tqdm(processed_questions_eng_lst):
  bow_corpus.append(processed_questions_eng_gensim.doc2bow(doc))

tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

100%|██████████| 2940542/2940542 [00:10<00:00, 271178.84it/s]


In [20]:
%%time

model_lst = []
for n_t in tqdm(np.linspace(5, 20, 5)):
  model_lst.append(gensim.models.LdaMulticore(tfidf_corpus, num_topics=n_t, id2word=processed_questions_eng_gensim, passes=5))

100%|██████████| 5/5 [31:40<00:00, 380.06s/it]

CPU times: user 29min 31s, sys: 3min 49s, total: 33min 21s
Wall time: 31min 40s


# visualize results

In [21]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_lst[0], tfidf_corpus, processed_questions_eng_gensim,  mds="mmds")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.138763  0.181768       1        1  22.559307
2     -0.005807 -0.233850       2        1  20.899364
1     -0.212793 -0.077966       3        1  19.387379
0     -0.125790  0.216274       4        1  18.649492
3      0.205628 -0.086225       5        1  18.504457, topic_info=        Term          Freq         Total Category  logprob  loglift
219     soil  24005.000000  24005.000000  Default  30.0000  30.0000
25     plant  69154.000000  69154.000000  Default  29.0000  29.0000
52     price  25957.000000  25957.000000  Default  28.0000  28.0000
1160   reply  18121.000000  18121.000000  Default  27.0000  27.0000
1130  follow  17186.000000  17186.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
82      type   7893.577896  42730.442204   Topic5  -4.9228  -0.0017
21     start   5590.266518  23547.040475   Topic5  -5.2678   0.2492
25     plant   6668.661288  69154.151970   Topic5  -5.0914  -0.6518
36      want   4683.992932  26770.995139   Topic5  -5.4447  -0.0560
149     good   4522.170266  30421.349533   Topic5  -5.4798  -0.2190

[484 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
745       1  0.726198  acre
745       2  0.019008  acre
745       3  0.107554  acre
745       4  0.039787  acre
745       5  0.107554  acre
...     ...       ...   ...
1035      1  0.000441  zero
1035      2  0.001766  zero
1035      3  0.006179  zero
1035      4  0.001324  zero
1035      5  0.990036  zero

[1622 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])

In [22]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_lst[1], tfidf_corpus, processed_questions_eng_gensim,  mds="mmds")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.275520  0.066554       1        1  14.240920
4      0.159729  0.225348       2        1  13.748477
2      0.246967 -0.094639       3        1  13.102917
7      0.152347  0.050088       4        1  12.967907
1     -0.192061 -0.203656       5        1  11.945348
5     -0.092440  0.223912       6        1  11.755907
0      0.088733 -0.259625       7        1  11.676590
6     -0.087755 -0.007982       8        1  10.561935, topic_info=          Term          Freq         Total Category  logprob  loglift
1160     reply  18281.000000  18281.000000  Default  30.0000  30.0000
1109  response  15679.000000  15679.000000  Default  29.0000  29.0000
489       weed  15834.000000  15834.000000  Default  28.0000  28.0000
1130    follow  17353.000000  17353.000000  Default  27.0000  27.0000
116       milk  20693.000000  20693.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
275      cause   3236.895614  24347.544277   Topic8  -5.2534   0.2301
28        grow   3323.229483  36898.931046   Topic8  -5.2271  -0.1593
161       farm   2945.198909  41257.992087   Topic8  -5.3479  -0.3918
52       price   2830.620214  26134.040438   Topic8  -5.3876   0.0252
525        wat   2639.271704  18275.494157   Topic8  -5.4576   0.3129

[771 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1070      1  0.001268  abortion
1070      3  0.002536  abortion
1070      5  0.002536  abortion
1070      6  0.001268  abortion
1070      8  0.991715  abortion
...     ...       ...       ...
1035      4  0.002192      zero
1035      5  0.099081      zero
1035      6  0.001754      zero
1035      7  0.892171      zero
1035      8  0.000438      zero

[3540 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 8, 2, 6, 1, 7])

In [23]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_lst[2], tfidf_corpus, processed_questions_eng_gensim,  mds="mmds")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.147031  0.019570       1        1  12.711583
3     -0.058139 -0.276516       2        1  10.148035
6     -0.218797 -0.084504       3        1   8.616412
1     -0.190679  0.197556       4        1   8.295648
10     0.251182 -0.126719       5        1   8.119564
7      0.087416 -0.202178       6        1   7.941159
9      0.025730  0.257255       7        1   7.927342
11    -0.137308 -0.185848       8        1   7.825039
4     -0.286315  0.054948       9        1   7.693495
8      0.150154  0.208033      10        1   7.286046
2      0.277208  0.053751      11        1   6.955631
0     -0.047482  0.084652      12        1   6.480046, topic_info=          Term          Freq         Total Category  logprob  loglift
1160     reply  18863.000000  18863.000000  Default  30.0000  30.0000
1109  response  16192.000000  16192.000000  Default  29.0000  29.0000
1130    follow  17914.000000  17914.000000  Default  28.0000  28.0000
489       weed  15765.000000  15765.000000  Default  27.0000  27.0000
73      banana  23023.000000  23023.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
82        type   1709.919269  44214.674665  Topic12  -5.4031  -0.5162
25       plant   1663.752518  72740.946219  Topic12  -5.4305  -1.0414
98     disease   1448.502285  16636.240972  Topic12  -5.5690   0.2954
35         use   1417.395445  37166.628662  Topic12  -5.5907  -0.5301
161       farm   1386.033870  41271.141851  Topic12  -5.6131  -0.6573

[1099 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
4668      2  0.989893      abag
6119      3  0.991074    abasin
6119      5  0.003490    abasin
1070      4  0.001354  abortion
1070      6  0.002707  abortion
...     ...       ...       ...
1035      8  0.000395      zero
1035      9  0.001973      zero
1035     10  0.000789      zero
1035     11  0.000395      zero
1035     12  0.989474      zero

[6441 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 7, 2, 11, 8, 10, 12, 5, 9, 3, 1])

In [24]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_lst[3], tfidf_corpus, processed_questions_eng_gensim,  mds="mmds")

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.274786 -0.018139       1        1  9.414982
11     0.336244 -0.055970       2        1  8.977040
1      0.265919 -0.216435       3        1  7.892813
8      0.278299  0.073968       4        1  7.720711
0      0.204367  0.218422       5        1  6.204942
12     0.071885 -0.241365       6        1  5.840899
5     -0.259872 -0.170025       7        1  5.829989
4     -0.104580 -0.198428       8        1  5.800545
7     -0.042280  0.119182       9        1  5.769842
10    -0.274990  0.105276      10        1  5.658700
9      0.121243 -0.011438      11        1  5.505140
6     -0.060169 -0.316456      12        1  5.428186
14     0.077869  0.246720      13        1  5.365332
15    -0.060921  0.309254      14        1  5.137529
2     -0.198148  0.219068      15        1  4.807080
3     -0.080079 -0.063634      16        1  4.646270, topic_info=          Term          Freq         Total Category  logprob  loglift
1160     reply  19536.000000  19536.000000  Default  30.0000  30.0000
1109  response  16757.000000  16757.000000  Default  29.0000  29.0000
1130    follow  18493.000000  18493.000000  Default  28.0000  28.0000
690      space  14193.000000  14193.000000  Default  27.0000  27.0000
89       fruit  16365.000000  16365.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
35         use   1637.541789  36546.995744  Topic16  -5.1137  -0.0363
189     cattle   1295.190283  13156.604215  Topic16  -5.3482   0.7508
161       farm   1415.058639  43185.064448  Topic16  -5.2597  -0.3492
82        type   1383.061665  44407.354893  Topic16  -5.2826  -0.4000
557    farmers   1200.655977  11337.644759  Topic16  -5.4240   0.8238

[1291 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4668      2  0.992116     abag
547       7  0.001107  abanana
547       8  0.995363  abanana
547      13  0.002214  abanana
6119      1  0.993363   abasin
...     ...       ...      ...
1035      8  0.000387     zero
1035      9  0.000775     zero
1035     13  0.000387     zero
1035     14  0.994894     zero
1035     16  0.000775     zero

[8137 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 12, 2, 9, 1, 13, 6, 5, 8, 11, 10, 7, 15, 16, 3, 4])

In [25]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_lst[4], tfidf_corpus, processed_questions_eng_gensim,  mds="mmds")

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.221031  0.272087       1        1  6.914294
13     0.082829 -0.015894       2        1  6.746502
0      0.004476  0.362682       3        1  6.076888
17    -0.105728 -0.052522       4        1  5.453720
12    -0.320177  0.156954       5        1  5.359091
2      0.029370  0.204883       6        1  5.305725
18     0.346463  0.046558       7        1  5.250869
8     -0.124216  0.150046       8        1  5.102238
6      0.288059 -0.084801       9        1  5.084881
10     0.163602  0.115532      10        1  5.046893
9     -0.354222 -0.086318      11        1  4.992237
7     -0.174108 -0.323841      12        1  4.939253
19    -0.261220  0.022764      13        1  4.926455
4      0.131848 -0.223069      14        1  4.694025
15     0.276038 -0.194030      15        1  4.401287
1      0.274512  0.172831      16        1  4.171408
5      0.166017  0.273273      17        1  4.094535
16     0.047974 -0.361499      18        1  4.004267
11    -0.230512 -0.195637      19        1  3.728033
14    -0.019974 -0.240000      20        1  3.707400, topic_info=          Term          Freq         Total Category  logprob  loglift
1109  response  18313.000000  18313.000000  Default  30.0000  30.0000
5       wefarm  23891.000000  23891.000000  Default  29.0000  29.0000
489       weed  19113.000000  19113.000000  Default  28.0000  28.0000
1130    follow  20203.000000  20203.000000  Default  27.0000  27.0000
10      rabbit  24075.000000  24075.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
79        best   2260.574827  60667.017474  Topic20  -4.5655   0.0051
42      county   1765.068827  13203.671117  Topic20  -4.8129   1.2825
126      maize   1794.842477  63522.371274  Topic20  -4.7962  -0.2716
28        grow   1660.238129  40421.006239  Topic20  -4.8742   0.1025
23        seed   1545.812772  31957.518270  Topic20  -4.9456   0.2660

[1400 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
547      20  0.999424  abanana
3150     11  0.996975    abaut
173       2  0.000460     able
173       3  0.001840     able
173       5  0.011041     able
...     ...       ...      ...
1035      5  0.002501     zero
1035      6  0.963852     zero
1035      9  0.032101     zero
1035     10  0.000417     zero
1035     20  0.000834     zero

[7276 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 14, 1, 18, 13, 3, 19, 9, 7, 11, 10, 8, 20, 5, 16, 2, 6, 17, 12, 15])